In [1]:
from skimage.feature import hog
from skimage.transform import pyramid_gaussian
from skimage.io import imread
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from skimage import color
from imutils.object_detection import non_max_suppression
import imutils
import numpy as np
import argparse
import cv2
import os
import glob
from PIL import Image # This will be used to read/modify images (can be done via OpenCV too)
from numpy import *
import pandas as pd

In [2]:
# define parameters of HOG feature extraction
orientations = 9
pixels_per_cell = (8, 8)
cells_per_block = (2, 2)
threshold = .3

In [3]:
pos_im_path=r'./smile/'
neg_im_path=r'./not-smile/'

In [4]:
pos_im_listing = os.listdir(pos_im_path) # it will read all the files in the positive image path (so all the required images)
neg_im_listing = os.listdir(neg_im_path)

In [5]:
num_pos_samples = size(pos_im_listing) # simply states the total no. of images
num_neg_samples = size(neg_im_listing)

In [6]:
# data= np.array([])
# labels =np.array([])
data=[]
labels=[]

In [7]:
# compute HOG features and label them:

for file in pos_im_listing: #this loop enables reading the files in the pos_im_listing variable one by one
    img = Image.open(pos_im_path + '\\' + file) # open the file
    #img = img.resize((64,128))
    gray = img.convert('L') # convert the image into single channel i.e. RGB to grayscale
    # calculate HOG for positive features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True)# fd= feature descriptor
    # print(np.array(fd))
    data.append(fd)
    labels.append(1)
    # data=np.append(data,fd)
    # labels=np.append(labels,1)

In [8]:
k=pd.DataFrame(data=data)

In [9]:
k

,0,1,2,3,4,5,6,7,8,9,...,250550,250551,250552,250553,250554,250555,250556,250557,250558,250559
0,0.017661,0.000000,0.001880,0.011944,0.123868,0.437784,0.002542,0.000000,0.002950,0.036288,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.670423,0.004275,0.007484,0.007708,0.007648,0.000000,0.000000,0.016300,0.162425,0.284653,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.139347,0.080889,0.348845,0.404093,0.433330,0.070610,0.018172,0.053185,0.044398,0.110431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.230051,0.052722,0.100717,0.117539,0.393767,0.235449,0.247790,0.031701,0.043941,0.146035,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,0.096994,0.111928,0.156087,0.134931,0.164034,0.030491,0.036870,0.028900,0.032995,0.145298,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2160,0.220796,0.121888,0.319566,0.434660,0.266659,0.027700,0.021508,0.000000,0.000000,0.129588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2161,0.017311,0.000000,0.000000,0.000000,0.027410,0.053732,0.075486,0.014879,0.009124,0.020374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2162,0.088654,0.121839,0.081443,0.040369,0.140795,0.056679,0.063093,0.052898,0.027972,0.083390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
k[17387].isnull().sum()

31

In [11]:
new_df=k.iloc[:,0:17388]

In [36]:
lb=labels[0:2164]

In [37]:
new_df['labels']=lb

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9568\1443792301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['labels']=lb


In [38]:
new_df

,0,1,2,3,4,5,6,7,8,9,...,17379,17380,17381,17382,17383,17384,17385,17386,17387,labels
0,0.017661,0.000000,0.001880,0.011944,0.123868,0.437784,0.002542,0.000000,0.002950,0.036288,...,0.377309,0.153595,0.202760,0.100277,0.077152,0.083398,0.098497,0.177060,0.068912,1
1,0.670423,0.004275,0.007484,0.007708,0.007648,0.000000,0.000000,0.016300,0.162425,0.284653,...,0.340594,0.156145,0.089773,0.124945,0.147951,0.187375,0.101103,0.034574,0.067670,1
2,0.139347,0.080889,0.348845,0.404093,0.433330,0.070610,0.018172,0.053185,0.044398,0.110431,...,0.040282,0.037808,0.050773,0.107816,0.185037,0.354914,0.384626,0.186820,0.039762,1
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
4,0.230051,0.052722,0.100717,0.117539,0.393767,0.235449,0.247790,0.031701,0.043941,0.146035,...,0.155371,0.108835,0.076629,0.060310,0.014257,0.025934,0.043490,0.183140,0.006441,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,0.096994,0.111928,0.156087,0.134931,0.164034,0.030491,0.036870,0.028900,0.032995,0.145298,...,0.621038,0.022856,0.060865,0.000000,0.000000,0.017497,0.000000,0.000000,0.024745,1
2160,0.220796,0.121888,0.319566,0.434660,0.266659,0.027700,0.021508,0.000000,0.000000,0.129588,...,0.028496,0.014045,0.014172,0.004248,0.013298,0.022337,0.039828,0.006850,0.006008,1
2161,0.017311,0.000000,0.000000,0.000000,0.027410,0.053732,0.075486,0.014879,0.009124,0.020374,...,0.481302,0.345633,0.253462,0.140618,0.059988,0.052521,0.005998,0.060747,0.013411,1
2162,0.088654,0.121839,0.081443,0.040369,0.140795,0.056679,0.063093,0.052898,0.027972,0.083390,...,0.219088,0.220455,0.194934,0.333163,0.197957,0.112954,0.047237,0.019937,0.069349,1


In [39]:
m=[]
labels_=[]

In [40]:
# Same for the negative images
for file in neg_im_listing:
    img= Image.open(neg_im_path + '\\' + file)
    #img = img.resize((64,128))
    gray= img.convert('L')
    # Now we calculate the HOG for negative features
    fd = hog(gray, orientations, pixels_per_cell, cells_per_block, block_norm='L2', feature_vector=True) 
    m.append(fd)
    labels_.append(0)
    # data=np.append(data,fd)
    # labels=np.append(labels,0)

In [14]:
# encode the labels, converting them from strings to integers
# le = LabelEncoder()
# labels = le.fit_transform(labels)

In [41]:
k2=pd.DataFrame(data=m)

In [42]:
k2

,0,1,2,3,4,5,6,7,8,9,...,167786,167787,167788,167789,167790,167791,167792,167793,167794,167795
0,0.273951,0.058340,0.202937,0.070423,0.195680,0.000000,0.184488,0.058340,0.000000,0.156544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.328503,0.071601,0.158445,0.187665,0.148323,0.177706,0.187566,0.192494,0.271485,0.241310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.188883,0.042236,0.120204,0.021118,0.151106,0.021118,0.106848,0.042236,0.000000,0.207771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.553411,0.000000,0.000000,0.000000,0.059828,0.033445,0.042305,0.000000,0.000000,0.104699,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.020005,0.004473,0.014146,0.002237,0.010003,0.002237,0.016975,0.004473,0.000000,0.053013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,0.363275,0.126607,0.025406,0.010041,0.074991,0.006227,0.011139,0.013210,0.112369,0.402312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1832,0.172883,0.027438,0.014419,0.044803,0.048067,0.155088,0.377362,0.301317,0.232500,0.126154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1833,0.071566,0.040006,0.050605,0.000000,0.590416,0.000000,0.000000,0.000000,0.000000,0.286263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1834,0.205271,0.116138,0.163405,0.123603,0.190468,0.275397,0.118633,0.102874,0.026346,0.257371,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
new_df2=k2.iloc[:,0:17388]

In [44]:
new_df2['labels']=labels_

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9568\4045406910.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2['labels']=labels_


In [45]:
new_df2

,0,1,2,3,4,5,6,7,8,9,...,17379,17380,17381,17382,17383,17384,17385,17386,17387,labels
0,0.273951,0.058340,0.202937,0.070423,0.195680,0.000000,0.184488,0.058340,0.000000,0.156544,...,0.301126,0.321592,0.105235,0.089258,0.034609,0.000000,0.044716,0.009601,0.051904,0
1,0.328503,0.071601,0.158445,0.187665,0.148323,0.177706,0.187566,0.192494,0.271485,0.241310,...,0.353080,0.000000,0.005829,0.000000,0.000000,0.031856,0.128774,0.119074,0.482703,0
2,0.188883,0.042236,0.120204,0.021118,0.151106,0.021118,0.106848,0.042236,0.000000,0.207771,...,0.222191,0.061807,0.104742,0.261928,0.264513,0.212929,0.127927,0.000000,0.000000,0
3,0.553411,0.000000,0.000000,0.000000,0.059828,0.033445,0.042305,0.000000,0.000000,0.104699,...,0.146918,0.260673,0.154128,0.118245,0.091128,0.027346,0.036498,0.037197,0.025903,0
4,0.020005,0.004473,0.014146,0.002237,0.010003,0.002237,0.016975,0.004473,0.000000,0.053013,...,0.243894,0.043051,0.092505,0.050427,0.031934,0.137522,0.103023,0.235472,0.087185,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831,0.363275,0.126607,0.025406,0.010041,0.074991,0.006227,0.011139,0.013210,0.112369,0.402312,...,0.184475,0.025461,0.000000,0.006232,0.005880,0.000000,0.016919,0.045161,0.183850,0
1832,0.172883,0.027438,0.014419,0.044803,0.048067,0.155088,0.377362,0.301317,0.232500,0.126154,...,0.114318,0.087393,0.060064,0.027594,0.014118,0.013139,0.020165,0.029639,0.021582,0
1833,0.071566,0.040006,0.050605,0.000000,0.590416,0.000000,0.000000,0.000000,0.000000,0.286263,...,0.346028,0.124953,0.095082,0.059791,0.024230,0.027090,0.198731,0.119790,0.132487,0
1834,0.205271,0.116138,0.163405,0.123603,0.190468,0.275397,0.118633,0.102874,0.026346,0.257371,...,0.149057,0.245532,0.110923,0.150537,0.105259,0.167852,0.092698,0.077399,0.052898,0


In [47]:
result=new_df.append(new_df2)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9568\2657074430.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=new_df.append(new_df2)


In [48]:
result.shape

(4000, 17389)

In [52]:
X=result.iloc[:,0:17388]

In [63]:
X.fillna(0,inplace=True)

In [64]:
y=result['labels']

In [65]:
(trainData, testData, trainLabels, testLabels) = train_test_split(
	X, y, test_size=0.20, random_state=42)

In [66]:
model = LinearSVC()
model.fit(trainData, trainLabels)

C:\Users\Lenovo\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [67]:
predictions = model.predict(testData)

In [68]:

print(classification_report(testLabels, predictions))

              precision    recall  f1-score   support

           0       0.61      0.65      0.63       354
           1       0.71      0.67      0.69       446

    accuracy                           0.66       800
   macro avg       0.66      0.66      0.66       800
weighted avg       0.66      0.66      0.66       800

